In [1]:
import pandas as pd
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

# 1. Análisis de personas distintas atendidas por diagnóstico

En este cuadernillo se quieren responder las siguiente preguntas:

1. ¿Cuántas personas fueron atendidas para cada uno de los diagnósticos del INT?


In [2]:
# Lee solamente la base del INT
ANIOS_A_ANALIZAR = [2016, 2017, 2018, 2019]

df_torax = (
    pl.scan_csv("../data/processed/egresos_procesados.csv")
    .filter(
        (pl.col("ESTABLECIMIENTO_SALUD") == 112103) & (pl.col("ANO_EGRESO").is_in(ANIOS_A_ANALIZAR))
    )
    .collect(streaming=True)
)

Los problemas al momento de calcular la cantidad de pacientes distintos para un problema de salud son los siguientes:

1. Un paciente puede tener egresos hospitalarios con distintos diagnosticos que apuntan a un mismo problema de salud (Ej: Diagnostico E840, E841, E849 apuntar al problema de salud "Fibrosis Quistica"). Esto es un problema, ya que el paciente sera considerado como un paciente distinto dependiendo del diagnostico asignado. A modo de ilustracion, se tiene el siguiente ejemplo:

   - Paciente X diagnosticado con E840 y E841 en un mismo año. Esto significaría que hay 1 paciente distinto para E840 y E841. Sin embargo, si analizamos el problema de salud "Fibrosis Quística", solamente existiría 1 paciente distinto (y no 2).

Debido a lo anterior, es necesario tomar las precauciones necesarios al hacer los calculos.


In [7]:
# Define diagnosticos para los problemas de salud de interes
PROBLEMAS_DE_SALUD = {
    "FIBROSIS_QUISTICA": ["E840", "E841", "E849", "E848"],  # Listo
    "HIPERTENSION_PULMONAR": ["I270", "I272"],  # Listo
    "FIBROSIS_PULMONAR": ["J841"],
    "ESCLERODERMIA": ["M348"],  # Listo
    "MARFAN": ["Q874"],  # listo
    "EXCAVATUM": ["Q676"],  # Listo
    "CARINATUM": ["Q677"],  # Listo
    "SILICOSIS": ["J60X", "J61X", "J628", "J65X", "J678", "J679"], # Listo
    "CARDIOPATIA_CONGENITA_ADULTO": [
        "Q201",
        "Q202",
        "Q203",
        "Q204",
        "Q205",
        "Q206",
        "Q208",
        "Q209",
        "Q210",
        "Q211",
        "Q212",
        "Q213",
        "Q214",
        "Q218",
        "Q220",
        "Q221",
        "Q222",
        "Q223",
        "Q224",
        "Q225",
        "Q228",
        "Q230",
        "Q231",
        "Q233",
        "Q240",
        "Q241",
        "Q244",
        "Q245",
        "Q246",
        "Q248",
        "Q249",
        "Q250",
        "Q251",
        "Q253",
        "Q254",
        "Q255",
        "Q256",
        "Q257",
        "Q258",
        "Q259",
    ],  # Listo
    "TRASPLANTE_DE_PULMON": [], # GRD
    "ECMO": [], # GRD
    "CANCER_DE_PULMON": [
        "C340",
        "C341",
        "C342",
        "C343",
        "C348",
        "C349",
    ], # Listo
    "CIRUGIA_CANCER_DE_PULMON": [], # GRD
    "TRASPLANTE_DE_CORAZON": [], # GRD
    "ENFERMEDAD_CORONARIA": ["I251"], # Listo
    "ANEURISMA_DE_LA_AORTA": ["I711", "I712", "I713", "I715"], # Listo
    "DISECCION_DE_LA_AORTA": ["I710"], # Listo
    "MARCAPASOS": [], # GRD
    "INSUFICIENCIA_MITRAL": ["I051", "I340"], # Listo
    "INSUFICIENCIA_AORTICA": ["I061", "I351"], # LIsto
    "APNEA_DEL_SUENO": ["G473"], # Listo
}

In [8]:
resultados = {}
for problema_de_salud, codigos_diagnostico in PROBLEMAS_DE_SALUD.items():
    # Filtra el df del torax solamente a los diagnosticos del problema
    df_diagnostico = df_torax.filter(pl.col("DIAG1").is_in(codigos_diagnostico))

    # Calcula metricas totales para el periodo analizado
    egresos_totales = df_diagnostico.shape[0]
    pacientes_totales = df_diagnostico.select(pl.col("ID_PACIENTE").n_unique()).item(0, 0)

    # Calcula metricas desagregadas por anios
    resumen_diagnostico_agrupado = (
        df_diagnostico.group_by(pl.col(["ANO_EGRESO"]))
        .agg(n_egresos=pl.col("DIAG1").count(), n_pacientes=pl.col("ID_PACIENTE").n_unique())
        .sort(pl.col("ANO_EGRESO"))
        .mean()
    )
    # Selecciona la primera fila y la segunda columna del resumen ("n_egresos")
    egresos_promedios = resumen_diagnostico_agrupado.item(0, 1)
    pacientes_promedio = resumen_diagnostico_agrupado.item(0, 2)

    # Genera el string resumen de metricas totales y promedio
    linea_resumen_egresos = f"{egresos_totales} ({egresos_promedios})"
    linea_resumen_pacientes = f"{pacientes_totales} ({pacientes_promedio})"

    # Guarda los strings resumen en un DataFrame
    resumen_problema_salud = pd.DataFrame(
        {
            "egresos_totales_2016_a_2019 (suma - promedio)": [linea_resumen_egresos],
            "personas_atendidas_2016_a_2019 (suma - promedio)": [linea_resumen_pacientes],
        },
        index=[problema_de_salud],
    )

    resultados[problema_de_salud] = resumen_problema_salud

In [9]:
resumen_total_diags = pd.concat(resultados.values())
display(resumen_total_diags)

,egresos_totales_2016_a_2019 (suma - promedio),personas_atendidas_2016_a_2019 (suma - promedio)
FIBROSIS_QUISTICA,193 (48.25),64 (27.75)
HIPERTENSION_PULMONAR,90 (22.5),72 (18.75)
FIBROSIS_PULMONAR,462 (115.5),362 (99.5)
ESCLERODERMIA,94 (23.5),67 (18.0)
MARFAN,1 (1.0),1 (1.0)
EXCAVATUM,42 (10.5),34 (9.0)
CARINATUM,13 (3.25),12 (3.0)
SILICOSIS,277 (69.25),201 (56.0)
CARDIOPATIA_CONGENITA_ADULTO,459 (114.75),373 (100.75)
TRASPLANTE_DE_PULMON,0 (None),0 (None)


In [10]:
resumen_total_diags.to_excel("../data/interim/resumen_area_influencia_hospitalizado.xlsx")